In [2]:
import sys
import os
import json
import pandas as pd
import logging
import argparse

In [25]:
def get_diversity_action(sample_meta, table, tree):
    import qiime2.plugins.diversity.actions as diversity_actions
    action_results = diversity_actions.core_metrics_phylogenetic(
        phylogeny=rooted_tree,
        table=table,
        sampling_depth=10000,
        metadata=sample_meta,
    )
    return action_results

def get_diversity_pcoa_results(action_results, outdir):
    from skbio.stats.distance import DistanceMatrix
    from skbio import OrdinationResults
    
    div_results.jaccard_distance_matrix.view(skbio.stats.distance.DistanceMatrix).to_data_frame()
    div_results.jaccard_pcoa_results.view(skbio.OrdinationResults).proportion_explained
    div_results.jaccard_pcoa_results.view(skbio.OrdinationResults).samples

In [92]:
def get_alpha_rarefaction(table, tree, outdir):
    outdict = {}
    from q2_diversity._alpha._visualizer import _compute_rarefaction_data
    # from q2_diversity._alpha._visualizer import _compute_summary
    # from q2_diversity._alpha._visualizer import _alpha_rarefaction_jsonp

    metrics = {'observed_features', 'shannon', 'faith_pd'}
    tmp_table = table.view(biom.Table)
    max_depth = max(tmp_table.sum(axis='sample'))
    min_detph = 1
    steps = 10
    iterations = 10
    tmp_tree = rooted_tree.view(skbio.TreeNode)
    div_data = _compute_rarefaction_data(tmp_table, min_depth, max_depth, steps, iterations, tmp_tree, metrics)
    for m, data in div_data.items():
        filename = os.path.join(outdir, f'{m}.csv')
        # jsonp_filename = f'{m}.jsonp'
        # n_df = _compute_summary(data, 'sample-id')
        # _alpha_rarefaction_jsonp(outdir, jsonp_filename, m, n_df, '')
        data.columns = [f'depth-{t[0]}_iter{t[1]}' for t in data.columns.values]
        data.to_csv(filename)
        outdict.update({f'alpha_rarefaction_{m}_csv':filename})
    return outdict

In [4]:
def get_diversity(table_qza, tree_qza, outdir):
    from qiime2 import Artifact
    from qiime2 import Metadata
    
    table = Artifact.load(table_qza)
    tree = Artifact.load(tree_qza)
    tmpmanifest = os.path.join(outdir, 'tmpmanifest')
    df_tmp = pd.Series(table.view(pd.DataFrame).index.to_list()).rename('sample-id')
    df_tmp = pd.concat([df_tmp, df_tmp.rename('sample')], axis=1)
    df_tmp.to_csv(tmpmanifest, sep='\t', index=False)
    sample_meta = Metadata.load(tmpmanifest)
    
    # table = Artifact.load('test_out/merged_out/merged_tab.qza')
    # tmpmanifest = ('test_out/merged_out/tmpmanifest')
    # df_tmp = pd.Series(table.view(pd.DataFrame).index.to_list()).rename('sample-id')
    # df_tmp = pd.concat([df_tmp, df_tmp.rename('sample')], axis=1)
    # df_tmp.to_csv(tmpmanifest, sep='\t', index=False)
    # sample_meta = Metadata.load(tmpmanifest)
    # rooted_tree = Artifact.load('test_out/merged_out/merged_rooted_tree.qza')
    
    outdict = {}
    action_result = get_diversity_action(sample_meta, table, tree)
    outdict = get_diversity_pcon_results(action_result, outdir)
    outdict.update(get_alpha_rarefaction(table, tree, outdir))